In [60]:
import pandas as pd
import glob
import utils
import json
from functools import reduce

DATA = "./geojson/"

In [61]:
files = glob.glob(f"{DATA}municipios*")

In [62]:
files

['./geojson/municipios_canarias.geojson',
 './geojson/municipios_peninsula.geojson']

In [63]:
with open(files[0], "r") as test:
    test2 = json.load(test)

In [64]:
with open(f"{DATA}/test.geojson", "w", encoding="utf-8") as fp:
    json.dump(test2, fp)

In [65]:
with open(f"{DATA}/test.geojson", "r") as test:
    test3 = json.load(test)
test2 == test3


True

In [66]:
with open(files[0], "r") as file:
    merged_json = json.load(file)

municipios = []
for file in files:
    with open(file, "r") as json_file:
        municipios.append(json.load(json_file)['features'])
        
merged_json['features'] = reduce(lambda a, b: a+b, municipios)

In [67]:
with open(f"{DATA}total_municipios.geojson", "w") as fp:
    json.dump(merged_json, fp)

In [75]:
df = pd.DataFrame.from_records([elem['properties'] for elem in merged_json['features']])

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8219 entries, 0 to 8218
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   comunidad_name            8219 non-null   object 
 1   comunidad_normalizedName  8219 non-null   object 
 2   normalizedName            8219 non-null   object 
 3   name                      8219 non-null   object 
 4   provincia_name            8219 non-null   object 
 5   comunidad_id              94 non-null     float64
 6   provincia_normalizedName  8219 non-null   object 
 7   provincia_id              8219 non-null   int64  
 8   id                        8219 non-null   int64  
 9   comunidad_id_indra        8219 non-null   int64  
 10  layer                     94 non-null     object 
 11  path                      94 non-null     object 
 12  fid                       8125 non-null   float64
 13  comunida_id               8125 non-null   float64
dtypes: float